In [38]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

train_data = pd.read_csv('pps_train.csv',index_col=0)
test_data = pd.read_csv('pps_test.csv',index_col=0)
train_target = pd.read_csv('target.csv',index_col=0)

In [43]:
pd.concat([train_data,test_data])

,Id,MSSubClass,LotFrontage,LotArea,Street,Alley,LotShape,LandSlope,OverallQual,OverallCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.730463,2.885846,5.831328,19.212182,0.730463,0.730463,1.540963,0.000000,2.440268,1.820334,...,0,0,0,1,0,0,0,0,1,0
1,1.194318,2.055642,6.221214,19.712205,0.730463,0.730463,1.540963,0.000000,2.259674,2.440268,...,0,0,0,1,0,0,0,0,1,0
2,1.540963,2.885846,5.914940,20.347241,0.730463,0.730463,0.000000,0.000000,2.440268,1.820334,...,0,0,0,1,0,0,0,0,1,0
3,1.820334,3.011340,5.684507,19.691553,0.730463,0.730463,0.000000,0.000000,2.440268,1.820334,...,0,0,0,1,1,0,0,0,0,0
4,2.055642,2.885846,6.314735,21.325160,0.730463,0.730463,0.000000,0.000000,2.602594,1.820334,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,15.394418,1.194318,3.932510,14.081426,0.730463,0.730463,1.540963,0.000000,1.820334,2.259674,...,0,0,0,1,0,0,0,0,1,0
2913,15.395553,1.194318,3.932510,14.013314,0.730463,0.730463,1.540963,0.000000,1.820334,1.820334,...,0,0,0,1,1,0,0,0,0,0
2914,15.396687,2.055642,7.620056,22.782058,0.730463,0.730463,1.540963,0.000000,2.055642,2.259674,...,0,0,0,1,1,0,0,0,0,0
2915,15.397821,3.340760,5.744420,20.046557,0.730463,0.730463,1.540963,0.000000,2.055642,1.820334,...,0,0,0,1,0,0,0,0,1,0


In [25]:
train_data.drop('Id',axis=1,inplace=True)
test_data.drop('Id',axis=1,inplace=True)

In [26]:
from sklearn.preprocessing import RobustScaler
rbs = RobustScaler()
train_data_rbs=rbs.fit_transform(train_data)
test_data_rbs=rbs.fit_transform(test_data)

from sklearn.model_selection import train_test_split,cross_val_score,KFold
kfold = KFold(n_splits=3,shuffle=True)
x_train,x_test,y_train,y_test = train_test_split(train_data_rbs,train_target)

from sklearn.metrics import mean_squared_error

In [27]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb


from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [28]:
lasso = Lasso(alpha =0.0005)

ENet = ElasticNet(alpha=0.0005, l1_ratio=.9)

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber')

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213,nthread = -1)

In [29]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_data_rbs)
    rmse= np.sqrt(-cross_val_score(model, train_data_rbs, train_target, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [30]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1115 (0.0074)



In [31]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1116 (0.0074)



In [32]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Kernel Ridge score: 0.1181 (0.0091)



In [33]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 0.1166 (0.0072)



In [34]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 0.1163 (0.0070)



In [35]:
from mlxtend.regressor import StackingRegressor

stacked_averaged_models = StackingRegressor(regressors = [ENet, GBoost, KRR, model_xgb],
                                                 meta_regressor = lasso)

score = rmsle_cv(stacked_averaged_models)

print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 0.1163 (0.0079)


In [36]:
ans=test_data.copy()
ans=ans.reset_index().rename(columns={'index':'Id'})

In [37]:
models = [lasso,ENet,KRR,GBoost,model_xgb,stacked_averaged_models]
for model in models:
    model.fit(train_data_rbs,train_target)
    y_pred = model.predict(test_data_rbs)
    ans['SalePrice'] = y_pred
    model_name=str(model).split('(')[0]
    ans.loc[:,['Id','SalePrice']].to_csv(f'trials_final/{model_name}_.csv',index=False)